# OnStove notebook

Conceptualization, Methodology & Original code: Babak Khavari and Camilo Ramirez

This is the OnStove notebook. The purpose of the notebook is to give users the ability to run through the analysis with example data and it can therefore act as a complement to the publication and read the docs documentation.

The notebook is divded into 4 major parts: 
* **Data processing** - In this part of the analysis different geospatial datasets are read and processed to be used in the analysis. The datasets from this step are saved on the users computer. For future runs on the same area of interest this step can consquently be skipped unless datasets are switched. 
* **Calibration** - In this part the area of interest is calibrated. Raster cells are classified as either urban or rural, the electrification rate in different cells are determined and the rates of differet cooking fuels across settlements are calibrated. The calibrated data is saved in .pkl-file   
* **Model run** - The net-benefit for different stoves are determined in different parts of the study area. Summaries of the results documenting the benefits and costs of each stove type across the entire study area are produced. The results are saved as .pkl-file.  
* **Visualization** - Visulizing and saving different maps related to the results.

Each part of the notebook is divided into several different cells and each cell is described more in depth.

In [146]:
import os, sys, requests, zipfile
sys.path.append("..")

In [147]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
%autoreload 2
from onstove.onstove import OnStove, DataProcessor
from onstove.layer import RasterLayer, VectorLayer
from onstove.raster import interpolate
import time

# Downloading example data for Ghana from the Mendeley database

**Downloading and saving the techno-economic specification file**

In [122]:
tech_specs = r""
response = requests.get(gis_data)
open("tech_specs.csv", "wb").write(response.content)

NameError: name 'gis_data' is not defined

**Downloading and saving the socio-economic specification file**

In [123]:
soc_specs = r""
response = requests.get(gis_data)
open("soc_specs.csv", "wb").write(response.content)

NameError: name 'gis_data' is not defined

**Downloading, saving and unzipping the GIS data**

In [ ]:
gis_data = r""
response = requests.get(gis_data)
open("gis_data.zip", "wb").write(response.content)

with zipfile.ZipFile("gis_data.zip","r") as zip_ref:
    zip_ref.extractall("gis_data")

# 1. Data processing

## 1.1. Create a data processor

This cell creates your dataprocessor. The dataprocessor is your model and will set the base for your model. OnStove is a raster-based model, hence the resolution becomes important. We specify the resolution when creating the dataprocessor together with the coordinate sytsem. This will ensure that all rasters are resampled to the correct resolution and all datasets (vectors and rasters) are reprojected to the target coordinate system.

In this example we use the pseudo-mercator coordinate system (EPSG:3857) and a spatial resolution of 1 sq. km. 

In [149]:
start = time.time()

country = OnStove()
data = DataProcessor(project_crs=3857, cell_size=(1000, 1000))
output_directory = '../example/results'
data.output_directory = output_directory

## 1.2. Create a data processor

In [150]:
path = os.path.join('..', 'example','GHA.csv')
country.read_scenario_data(path, delimiter=',')

## 1.3. Add a mask layer (country boundaries)

The mask layer dictates what falls within your area of interest and what is excluded from your analysis. For mask layer we use the administrative boundaries of the country.

For this a function called *add_mask_layer*. *add_mask_layer* takes four inputs:
1. `category`- referencing the cateogry of the layer
2. `name` - referencing the name of the layer
3. `layer_path` - from where to read the data
4. `postgres` - boolean determining indicating whether the data is saved on disc or in a PostgreSQL database. Default is `False`, meaning the dataset is saved on the disc.

In [151]:
adm_path = r"../example/Ghana/Administrative/Country_boundaries/Country_boundaries.geojson"
data.add_mask_layer(category='Administrative', name='Country_boundaries', layer_path=adm_path)

A raster base layer is needed to make every output match its grid and extent. For this, two additional options need to be passed to the `add_layer` method:
* `base_layer`: if `True` the added layer will be considered as the base layer. 
* `resample`: this is the resampling method to be used when resampling this layer to the desired `cell_size` if a `cell_size` is provided.

In [152]:
data.add_layer(category='Base', name='Base', layer_path=r"../example/Ghana/Forest/Forest.tif",
               layer_type='raster', base_layer=True, resample='nearest')

## 1.4. Add GIS layers

Similarly, we can add data layers using the `add_layer` method. A layer `name`, `layer_path` and `postgres` conection also need to be provided (the `postgres` conection defaults to `False`). In addition, the following arguments can be passed:
* `category`: this is used to group all datasets into a category in the final output, e.g. `demand` or `supply`. 
* `layer_type`: this argument is required with two possible options `raster` or `vector`, we should pass either one according to the dataset you are adding. 
* `resample`: this defines what resampling method to use when changing the resolution of the raster. The change of resolution happens when the layer gets aligned with the base layer.

### 1.4.1. Demographics

In [153]:
pop_path = r"../example/Ghana/Population\Population.tif"
data.add_layer(category='Demographics', name='Population', 
               layer_path=pop_path, layer_type='raster', resample='sum')

urban_path = r"../example/Ghana/Urban/Urban.tif"
data.add_layer(category='Demographics', name='Urban_rural_divide', 
               layer_path=urban_path, layer_type='raster', resample='nearest')

### 1.4.2. Biomass

In [154]:
forest_path = r"../example/Ghana/Forest/Forest.tif"
data.add_layer(category='Biomass', name='Forest', 
               layer_path=forest_path, layer_type='raster', resample='average')

friction_path = r"../example/Ghana/Friction/Friction.tif"
data.add_layer(category='Biomass', name='Friction', layer_path=friction_path, 
               layer_type='raster', resample='average')

### 1.4.3. Electricity

In [155]:
mv_path = r"../example/Ghana/MV lines/MV_lines.geojson"
data.add_layer(category='Electricity', name='MV_lines', 
               layer_path=mv_path, layer_type='vector')

ntl_path = r"../example/Ghana/Night time lights\Night_time_lights.tif"
data.add_layer(category='Electricity', name='Night_time_lights', 
               layer_path=ntl_path, layer_type='raster', resample='average')

### 1.4.4. LPG

In [156]:
lpg_path = r"../example/Ghana/Traveltime/Traveltime.tif"
data.add_layer(category='LPG', name='LPG Traveltime', 
               layer_path=lpg_path, layer_type='raster', resample='average')

### 1.4.5. Biogas

In [157]:
buffaloes = r"../example/Ghana/Livestock\buffaloes\buffaloes.tif"
cattles = r"../example/Ghana/Livestock\cattles\cattles.tif"
poultry = r"../example/Ghana/Livestock\poultry\poultry.tif"
goats = r"../example/Ghana/Livestock\goats\goats.tif"
pigs = r"../example/Ghana/Livestock\pigs\pigs.tif"
sheeps = r"../example/Ghana/Livestock\sheeps\sheeps.tif"

for key, path in {'buffaloes': buffaloes,
                  'cattles': cattles,
                  'poultry': poultry,
                  'goats': goats,
                  'pigs': pigs,
                  'sheeps': sheeps}.items():
    data.add_layer(category='Biogas/Livestock', name=key, layer_path=path,
                   layer_type='raster', resample='nearest', rescale=True)

In [158]:
temperature = r"../example/Ghana/Temperature\Temperature.tif"
data.add_layer(category='Biogas', name='Temperature', layer_path=temperature,
               layer_type='raster', resample='average')
data.layers['Biogas']['Temperature'].save(f'{data.output_directory}/Biogas/Temperature')

### 1.5. Mask reproject and align all required layers

In [159]:
data.mask_layers(datasets={'Demographics': ['Population', 'Urban_rural_divide'],
                           'Biomass': ['Forest', 'Friction'],
                           'Electricity': ['Night_time_lights'],
                           'LPG': ['LPG Traveltime'],
                           'Biogas': ['Temperature']})

In [160]:
data.align_layers(datasets='all')

In [161]:
data.reproject_layers(datasets={'Electricity': ['MV_lines']})

C:\Users\khavari\.conda\envs\cca\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [162]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

Execution time: 0 min 24 sec


# 2. Model preparation

## 2.1. Read the model data

In [163]:
path = os.path.join('..', 'example', 'GHA.csv')
country.read_scenario_data(path, delimiter=',')

## 2.2. Add a country mask layer

In [164]:
path = os.path.join('..', 'example','Ghana','Administrative','Country_boundaries', 'Country_boundaries.geojson')
mask_layer = VectorLayer('admin', 'adm_0', layer_path=path)
country.mask_layer = mask_layer

## 2.3. Add a population base layer

In [165]:
path = os.path.join(output_directory,'Demographics','Population', 'Population.tif')
country.add_layer(category='Demographics', name='Population', layer_path=path, layer_type='raster', base_layer=True)
country.population_to_dataframe()

## 2.4. Calibrate population and urban/rural split

In [166]:
country.calibrate_current_pop()

ghs_path = r"..\example\results\Demographics\Urban_rural_divide\Urban_rural_divide.tif"
country.calibrate_urban_current_and_future_GHS(ghs_path)

## 2.5. Add wealth index GIS data

In [167]:
wealth_index = r"..\example\Ghana\Relative wealth index\GHA_relative_wealth_index.csv"
country.extract_wealth_index(wealth_index, file_type="csv")

## 2.6. Calculate value of time 

In [168]:
country.get_value_of_time()

## 2.7. Read electricity network GIS layers

In [169]:
path = os.path.join(output_directory, 'Electricity', 'MV_lines', 'MV_lines.geojson')
mv_lines = VectorLayer('Electricity', 'MV_lines', layer_path=path)

## 2.8. Calculate distance to electricity infrastructure 

In [171]:
country.distance_to_electricity(mv_lines=mv_lines)

## 2.9. Add night time lights data

In [172]:
path = os.path.join(output_directory, 'Electricity', 'Night_time_lights', 'Night_time_lights.tif')
ntl = RasterLayer('Electricity', 'Night_time_lights', layer_path=path)

country.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

## 2.10 Calibrate current electrified population

In [173]:
country.current_elec()
country.final_elec()

print('Calibrated grid electrified population fraction:', country.gdf['Elec_pop_calib'].sum() / country.gdf['Calibrated_pop'].sum())

Calibrated grid electrified population fraction: 0.8495907
